In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import gym
import numpy
import onnx
import onnxruntime
from gym import wrappers
import os
from typing import Any, Dict, List, Optional
import glob
# from base.rl.ppo import PPO

In [14]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
pthfile = 'net_params.pth'
onnxpath = 'test_simpleclass.onnx'
print(torch.__version__)
print(onnx.__version__)
print(onnxruntime.__version__)

1.4.0
1.10.1
1.9.0


In [15]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2,2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
    # num_flat_features：计算张量x的总特征量（把每个数字都看出是一个特征，即特征总量）
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension(批大小维度)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features

In [16]:
def test():
    batch_size = 1
    model = Net()
    input_shape = (1, 32, 32)
    
    # Initialize model with the pretrained weights
    map_location = 'gpu' if torch.cuda.is_available() else 'cpu'
    loaded_model = torch.load(pthfile, map_location=map_location)
    model.load_state_dict(loaded_model, strict=False)
    
    # set the model to inference mode
    model.eval()
    
    # data type nchw
    x = torch.rand(batch_size, *input_shape)
    input_names = ['input']
    output_names = ['output']
    # # Export the model
    torch.onnx.export(model,               # model being run
                    x,                         # model input (or a tuple for multiple inputs)
                    onnxpath,   # where to save the model (can be a file or file-like object)
                    export_params=True,        # store the trained parameter weights inside the model file
                    opset_version=10,          # the ONNX version to export the model to
                    do_constant_folding=True,  # whether to execute constant folding for optimization
                    input_names = ['input'],   # the model's input names
                    output_names = ['output'], # the model's output names
                    dynamic_axes={'input' : {0 : 'batch_size'},    # variable lenght axes
                                  'output' : {0 : 'batch_size'}}
                     )

In [17]:
test()